In [57]:
# Uncomment this if you are using Colab

# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# import sys
# sys.path.append('/content/drive/My Drive/cs231n-project/')

In [58]:
import numpy as np 
import cvxpy as cp
import time
import matplotlib.pyplot as plt
from lib.utils import relative_error

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
dataset_name = "ImageNet_8x8_5c"
X_train = np.load(f'data/ImageNet_8x8/X_{dataset_name}_train.npy')
y_train = np.load(f'data/ImageNet_8x8/y_{dataset_name}_train.npy')
y_train -= 1 # labels start from one, but we want them to start from 0
X_test = np.load(f'data/ImageNet_8x8/X_{dataset_name}_test.npy')
y_test = np.load(f'data/ImageNet_8x8/y_{dataset_name}_test.npy')
y_test -= 1 # labels start from one, but we want them to start from 0
C = y_train.max() + 1 # number of classes
print(f'C: {C}')
print(f'X_train.shape = {X_train.shape}')
print(f'y_train.shape = {y_train.shape}')
print(f'X_test.shape = {X_test.shape}')
print(f'y_test.shape = {y_test.shape}')

def flatten(x):
    return x.reshape((x.shape[0], -1))

NUM_TRAIN = 500
NUM_TEST = 100
X_train = flatten(X_train)[:NUM_TRAIN]
y_train = y_train[:NUM_TRAIN]
X_test = flatten(X_test)[:NUM_TEST]
y_test = y_test[:NUM_TEST]
print(f'X_train.shape = {X_train.shape}')
print(f'y_train.shape = {y_train.shape}')
print(f'X_test.shape = {X_test.shape}')
print(f'y_test.shape = {y_test.shape}')

C: 5
X_train.shape = (6500, 8, 8)
y_train.shape = (6500,)
X_test.shape = (250, 8, 8)
y_test.shape = (250,)
X_train.shape = (500, 64)
y_train.shape = (500,)
X_test.shape = (100, 64)
y_test.shape = (100,)


In [60]:
from lib.convex_fc_vector import Convex_FC_Vector_Solver, Convex_FC_Vector_Solver_Vectorized

In [61]:
solver = Convex_FC_Vector_Solver_Vectorized()
Z, Z_prime, loss = solver.solve(X_train, y_train, C, beta=0, max_iters=50000, verbose=True)



Y.shape = (500, 5)
X_V.shape = (500, 4161)
Z_V.shape = (4161, 5)
########
Started Convex FC Vector Solver...
                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 11 10:15:31 PM: Your problem has 41610 variables, 20 constraints, and 0 parameters.
(CVXPY) May 11 10:15:31 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 11 10:15:31 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 11 10:15:31 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 11 10:15:31 PM: Compiling problem (targ

In [62]:
# TRAIN
Y_hat = solver.predict(X_train, Z, Z_prime)
y_pred = np.argmax(Y_hat, axis=1)
train_accuracy = np.sum(y_train == y_pred) / y_train.shape[0]
print(f'train_accuracy: {train_accuracy}, loss_vec: {loss}')

# TEST
Y_hat_test = solver.predict(X_test, Z, Z_prime)
y_pred_test = np.argmax(Y_hat_test, axis=1)
test_accuracy = np.sum(y_test == y_pred_test) / y_test.shape[0]
print(f'test_accuracy: {test_accuracy}')

train_accuracy: 1.0, loss_vec: 3.734192662326168e-16
test_accuracy: 0.23


In [63]:
def compute_number_of_neurons(Z, Z_prime, tolerance):
    count = 0
    assert len(Z) == len(Z_prime)
    for k in range(len(Z)):
        eigvals1, _ = np.linalg.eig(Z[k])
        eigvals2, _ = np.linalg.eig(Z_prime[k])
        count += np.sum(eigvals1 > tolerance) + np.sum(eigvals2 > tolerance)
    return count

print(f'num_neurons: {compute_number_of_neurons(Z_vec, Z_prime_vec, tolerance=1e-6)}')

num_neurons: 605
